In [99]:
import json
import csv
from tqdm import tqdm

In [100]:
def modify_music(s):
    d = dict()
    key_index = 0
    value_index = 0
    current_index = 0
    check = 0
    modify = s[1:-1].replace("'",'')
    modify = ', '+modify+','
    for i in modify:
        if i == ":":
            key_index = current_index
        elif i == "," and check != 0:
            d[modify[value_index+2:key_index]] = modify[key_index+2:current_index]
            value_index = current_index
        check+=1
    
        current_index += 1
    return d

In [101]:
    # setup an array for writing each row in the csv file
    rows = []
    # extract fields from business json data set #
    # setup an array for storing each json entry
    business_data = []
    # setup an array for headers we are not using strictly
    business_header_removals = ['attributes', 'categories', 'hours', 'neighborhoods', 'open']
    # setup an array for headers we are adding
    business_header_additions = [
    'Sunday_Open', 'Sunday_Close', 'Monday_Open', 'Monday_Close', 'Tuesday_Open','Tuesday_Close', 'Wednesday_Open', 
    'Wednesday_Close', 'Thursday_Open','Thursday_Close', 'Friday_Open', 'Friday_Close', 'Saturday_Open', 
    'Saturday_Close','Noise Level', 'Attire', 'Alcohol', 'Price_Range', 'Delivery', 'Outdoor_Seating','Drive-Thru', 
    'Good_for_Groups', 'Has_TV', 'Caters', 'Waiter_Service','Good_for_Kids', 'Accepts_Credit_Cards', 
    'Takes_Reservations', 'Wi_Fi', 'Happy_Hour','Good_for_Dancing', 'Smoking', 'BYOB', 'Corkage', 'Take_Out', 
    'Coat_Check','Parking_Street', 'Parking_Valet', 'Parking_Lot', 'Parking_Garage','Parking_Validated', 'Music_DJ', 
    'Music_Karaoke', 'Music_Video', 'Music_Live','Music_Jukebox', 'Music_Background_Music', 'Is_Restaurants', 
    'Sandwiches', 'Fast Food','Nightlife', 'Pizza', 'Bars', 'Mexican', 'Food', 'American (Traditional)','Burgers',
    'Chinese', 'Italian', 'American (New)', 'Breakfast & Brunch', 'Thai','Indian', 'Sushi Bars', 'Korean',
    'Mediterranean', 'Japanese', 'Seafood', 'Middle Eastern', 'Pakistani', 'Barbeque', 'Vietnamese', 'Asian Fusion',
    'Diners','Greek', 'Vegetarian']
    # open the business source file
    with open('yelp_dataset/business.json') as f:
        # for each line in the json file
        for line in f:
            # store the line in the array for manipulation
            business_data.append(json.loads(line))
    # close the reader
    f.close()
    # append the initial keys as csv headers
    header = sorted(business_data[0].keys())
    # remove keys from the business data that we are not using strictly
    for headers in business_header_removals:
        if headers in header:
            header.remove(headers)
    # append the additional business related csv headers
    for headers in business_header_additions:
        header.append(headers)
    print('processing data in the business dataset...')
    # for every entry in the business data array
    for entry in tqdm(range(0, len(business_data))):
        row = []
        row.append(business_data[entry]['address'])
        row.append(business_data[entry]['business_id'])
        row.append(business_data[entry]['city'])
        row.append(business_data[entry]['is_open'])
        row.append(business_data[entry]['latitude'])
        row.append(business_data[entry]['longitude'])
        row.append(business_data[entry]['name'])
        row.append(business_data[entry]['postal_code'])
        row.append(business_data[entry]['review_count'])
        row.append(business_data[entry]['stars'])
        row.append(business_data[entry]['state'])

        
        # set up an array for the days of the week
        days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
        # iterate through the days of the week to extract the open and close times
        for time in days_of_week:
            # if a time is available
            if business_data[entry]['hours'] != None and time in business_data[entry]['hours']:
                index0 = business_data[entry]['hours'][time].find(":")
                index = business_data[entry]['hours'][time].find("-")
                # append the open time
                open_time = business_data[entry]['hours'][time][0:index0]
                row.append(int(open_time))
                # append the closing time
                close_time = business_data[entry]['hours'][time][index:index+2]
                close_time = close_time.replace("-",'')
                row.append(int(close_time))
            # else if a time is not available
            else:
                # append NA for the open time
                row.append(0)
                # append NA for the closing time
                row.append(0)

        # extract the attributes of interest
        attributes = ['Noise Level', 'Attire', 'Alcohol', 'Price Range', 'Delivery', 'Outdoor Seating', 'Drive-Thru',
                      'Good For Groups', 'Has TV', 'Caters', 'Waiter Service', 'Good for Kids', 'Accepts Credit Cards',
                      'Takes Reservations', 'Wi-Fi', 'Happy Hour', 'Good For Dancing', 'Smoking', 'BYOB', 'Corkage',
                      'Take-out', 'Coat Check']
        # for each attribute that is not nested
        for attribute in attributes:
            # if there is an attribute
            if business_data[entry]['attributes']!= None and attribute in business_data[entry]['attributes']:
                # if the attribute contains true
                if business_data[entry]['attributes'][attribute] is True:
                    row.append(1)
                # else if the attribute contains false
                elif business_data[entry]['attributes'][attribute] is False:
                    row.append(0)
                # else if the attribute is non-empty and not true of false
                elif business_data[entry]['attributes'][attribute] is not None:
                    row.append(business_data[entry]['attributes'][attribute])
            # else of the attribute is not available
            else:
                # append NA for the attribute
                row.append('NA')
        # extract the parking attributes
        
        
    
        parking_attributes = ['street', 'valet', 'lot', 'garage', 'validated']
        # for each parking attribute
        for attribute in parking_attributes:
            # if there are parking attributes
            if business_data[entry]['attributes'] != None and 'Parking' in business_data[entry]['attributes']:
                # if the parking attribute exists
                if attribute in business_data[entry]['attributes']['Parking']:
                    # if the parking attribute is true
                    if business_data[entry]['attributes']['Parking'][attribute] is True:
                        row.append(1)
                    # if the parking attribute is false
                    elif business_data[entry]['attributes']['Parking'][attribute] is False:
                        row.append(0)
                    # note that the parking attributes are all true/false so no need for is not None elif
                # else if the specific attribute is not available
                else:
                    row.append(0.5)
            # else if the parking attribute is not available
            else:
                row.append(0)
        # extract the music attributes
        music_attributes = ['dj', 'karaoke', 'video', 'live', 'jukebox', 'background_music']
        # for each music attribute
        for attribute in music_attributes:
            # if there are music attributes
            if business_data[entry]['attributes'] != None and 'Music' in business_data[entry]['attributes']:
                # if the music attribute exists
                d = modify_music(business_data[entry]['attributes']['Music'])
                if attribute in business_data[entry]['attributes']['Music']:
                    # if the music attribute is true
                    if d[attribute] == 'True':
                        row.append(1)
                    # if the music attribute is false
                    elif d[attribute] == 'False':
                        row.append(0)
                    # note that the music attributes are all true/false so no need for is not None elif
                # else if the specific attribute is not available
                else:
                    row.append(0.5)
            # else if the music attribute is not available
            else:
                row.append(0)

        # extract the categories
        categories_of_interest = ['Restaurants', 'Sandwiches', 'Fast Food', 'Nightlife', 'Pizza', 'Bars', 'Mexican',
                                  'Food', 'American (Traditional)', 'Burgers', 'Chinese', 'Italian',
                                  'American (New)', 'Breakfast & Brunch', 'Thai', 'Indian', 'Sushi Bars', 'Korean',
                                  'Mediterranean', 'Japanese', 'Seafood', 'Middle Eastern', 'Pakistani', 'Barbeque',
                                  'Vietnamese', 'Asian Fusion', 'Diners', 'Greek', 'Vegetarian']
        # for each category of interest
        for category in categories_of_interest:
            # if the category is in the category entry
            if business_data[entry]['categories'] != None and category in business_data[entry]['categories']:
                row.append(1)
            # else if the category is not in the entry
            else:
                row.append(0)
        # remove stray text, such as "\n" form address
        # set up an array for the cleaned row entries
        row_clean = []
        # for every item in the row
        for item in row:
            # scan and replace for nasty text
            row_clean.append(str(item).replace('\n', ' '))
        # after all fields have been extracted and cleaned, append the row to the rows array for writing to csv
        rows.append(row_clean)

  1%|          | 1811/192609 [00:00<00:20, 9216.75it/s]

processing data in the business dataset...


100%|██████████| 192609/192609 [00:20<00:00, 9543.34it/s] 


In [102]:
with open('yelp.csv', 'w') as out:
    writer = csv.writer(out)
    writer.writerow(header)
    for entry in tqdm(range(0, len(rows))):
        try:
            writer.writerow(rows[entry])
        except UnicodeEncodeError:
            continue
out.close()

100%|██████████| 192609/192609 [00:01<00:00, 108485.41it/s]
